## CARICAMENTO LIBRERIE E DATASET

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xvf /content/drive/MyDrive/labeled.tgz -C /content/

In [3]:
SEED = 9246

import os
import json
import time
from datetime import datetime
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import models

from PIL import Image

from skimage import io
import pandas as pd
import numpy as np
np.random.seed(SEED)

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import classification_report

import torchsummary
import random
import cv2

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")



In [ ]:
DATASET_DIR = '/content/progetto_2021_dataset'
DRIVE_FOLDER = '/content/drive/MyDrive/progetto_2021/model_checkpoint3'
DATASET_UNLABELED = '/content/progetto_2021_dataset/unlabeled1'
DATASET_UNLABELED2 = '/content/progetto_2021_dataset/unlabeled2'

JSON_DATA = os.path.join(DATASET_DIR, 'train_test_split_dict.json')

with open(JSON_DATA) as fp:
    dataset_json = json.load(fp)
    
len(dataset_json)

2

In [ ]:
labels = set()

for k in dataset_json.values():
    for lable_list in k.values():
        for v in lable_list:
            labels.add(v)
            
label_idx = {v: i for i, v in enumerate(sorted(labels))}

In [ ]:
import time

class VideoDataset(Dataset):

    def load_images_from_folder(self,folder):
      images = []
      for filename in os.listdir(folder):
          img = cv2.imread(os.path.join(folder,filename))
          if img is not None:
              images.append(img)
      return images

    def __init__(self, dataset_folder, labels_dict, transform=None, limit=30):
        """
        Args:
            dataset_folder (string): Path to the folder with mp4 files.
            labels_dict (dict): dict filename - list of label.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels_dict = labels_dict
        self.root_dir = dataset_folder
        self.limit = limit
        self.transform = transform or torchvision.transforms.ToTensor()
        
        self._files = np.array(list(self.labels_dict.keys()))

    def __len__(self):
        return len(self.labels_dict)

    def __getitem__(self, idx):
        name = self._files[idx]
        
        x = torch.zeros(self.limit, 3, 224, 224)
        #folder_pattern = os.path.join(self.root_dir, name,'*.png')
        #images = io.imread_collection(folder_pattern)

        folder = os.path.join(self.root_dir,name)
        images = self.load_images_from_folder(folder)    


        
        j = 0     
        for i, image in enumerate(images):
            if i < self.limit:
                j += 1
                image = Image.fromarray(image)
                image = self.transform(image)
                x[i] = image.unsqueeze(0)

        #if j < self.limit :
            #immagini = self.limit - j
            #for  k in range(immagini):
                #rand = random.randint(0,j-1)
                #image = images[rand]
                #image = Image.fromarray(image)
                #image = self.transform(image)
                #x[k+j] = image.unsqueeze(0)
      


        labels = torch.zeros(len(label_idx), dtype=torch.float32)
        for label in self.labels_dict[name]:
            labels[label_idx[label]] = 1
        
        return x, labels


transformations = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(224),
                                                  torchvision.transforms.ToTensor(),
                                                  #torchvision.transforms.RandomAffine(30),
                                                  #torchvision.transforms.RandomHorizontalFlip()
                                                  ])

datasetTrain = VideoDataset(DATASET_DIR, dataset_json['train'], transformations,limit=20)
datasetTest = VideoDataset(DATASET_DIR, dataset_json['test'], transformations,limit=20)

In [ ]:
batch_size = 16
trainingDataLoader = torch.utils.data.DataLoader(datasetTrain, 
                                                 batch_size=batch_size, 
                                                 shuffle=True, 
                                                 num_workers=2,
                                                 drop_last=True)
testDataLoader = torch.utils.data.DataLoader(datasetTest, 
                                             batch_size=batch_size,
                                             shuffle=True,
                                             drop_last=True, 
                                             num_workers=2)

#UnlabeledDataLoader = torch.utils.data.DataLoader(datasetUnlabeled, 
 #                                            batch_size=batch_size,
 #                                            shuffle=True,
 #                                            drop_last=True, 
 #                                            num_workers=2)
#Unlabeled2DataLoader = torch.utils.data.DataLoader(datasetUnlabeled2, 
 #                                            batch_size=batch_size,
  #                                           shuffle=True,
   #                                          drop_last=True, 
    #                                         num_workers=2)

## MODELLO

In [ ]:
import torchvision.models as models

resnet = models.vgg19(pretrained = True)

for params in resnet.parameters():
  params.requires_grad = False

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return x

resnet.fc = Identity()

In [4]:
dense = nn.Sequential(
    nn.Conv1d(20,20,kernel_size=(3),padding=1,stride=2),
    nn.ReLU(),
    nn.BatchNorm1d(20),
    nn.Conv1d(20,32,kernel_size=(5),padding=1,stride=2),
    nn.ReLU(),
    nn.BatchNorm1d(32),
    nn.Conv1d(32,64,kernel_size=(5),padding=1,stride=2),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.MaxPool1d(2),
    nn.Dropout(0.5),
    nn.Conv1d(64,64,kernel_size=(7),padding=1,stride=2),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Conv1d(64,64,kernel_size=(7),padding=1,stride=2),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.MaxPool1d(2),
    nn.Dropout(0.5),
    nn.Flatten(),
    nn.Linear(384,85),
    nn.Sigmoid()
)

In [ ]:
class SGNet(nn.Module):
    def __init__(self,resnet,dense):
      super().__init__()
      self.resnet = resnet
      self.dense = dense

    def forward(self,x):
      batch = torch.zeros((16,20,1000))
      i = 0
      for trailer in x:
          output = self.resnet(trailer) 
          batch[i] = output
          i +=1
      result = self.dense(batch.to(device))
      return result  

In [ ]:
model = SGNet(resnet,dense)
model = model.to(device)

## SALVATAGGIO E CARICAMENTO MODELLO

In [ ]:
CHECKPOINT = os.path.join(DRIVE_FOLDER, 'model.checkpoint')
MODELFILE = os.path.join(DRIVE_FOLDER, 'model.pth')

def save_checkpoint(epoch, model, optimizer, loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, CHECKPOINT)

def load_checkpoint(model, optimizer):
    if not os.path.exists(CHECKPOINT):
        return None, None
    checkpoint = torch.load(CHECKPOINT)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return epoch, loss

def save_model(model):
    torch.save(model.state_dict(), MODELFILE)

def load_lodel(model):
    if os.path.exists(MODELFILE):
        model.load_state_dict(torch.load(MODELFILE))

In [ ]:
load_lodel(model)

## DEFINIZIONE LOSS E OTTIMIZZATORE

In [ ]:
#Loss
#weight = torch.tensor(weight).to(device)
criterion = nn.BCELoss()

#optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

epoca = 0

## FASE DI TRAINING 

In [ ]:
epoca, error = load_checkpoint(model,optimizer)

In [ ]:
epochs = 50
train_losses = []

last_epoch = epoca

print_every = 20

try:
    for epoch in range(last_epoch or 0, epochs):
        print('Start epoch', epoch+1)
        model.train()
        running_loss = 0
        steps = 0
        for x, y in trainingDataLoader:

            steps += 1
            if epoch == 0 and steps == 1:
                print(f'input shape is {x.shape}, labels are {y.shape}')


            x = x.view(16,20,3,224,224)
            optimizer.zero_grad()
            logit = model.forward(x.to(device))

            loss = criterion(logit, y.to(device))
            loss.backward()

            optimizer.step()
            running_loss += loss.item()
            
            # TRAIN unlabeled batch
            # ...
            
            if steps % print_every == 0:
                print(f"epoch {epoch+1}/{epochs} "
                      f"train loss: {running_loss/steps:.6f} Labeled")

                save_checkpoint(epoch, model, optimizer, loss)

        save_model(model)
        train_losses.append(running_loss/steps)
        print(f"END Epoch {epoch+1}/{epochs} "
              f"Train loss: {running_loss/steps:.6f} ")
        
except KeyboardInterrupt: 
    print('Exiting from training early')
  

Start epoch 18
epoch 18/50 train loss: 0.119538 Labeled
epoch 18/50 train loss: 0.117859 Labeled
epoch 18/50 train loss: 0.117531 Labeled
epoch 18/50 train loss: 0.116856 Labeled
epoch 18/50 train loss: 0.116823 Labeled
epoch 18/50 train loss: 0.115882 Labeled
epoch 18/50 train loss: 0.115941 Labeled
epoch 18/50 train loss: 0.115530 Labeled
epoch 18/50 train loss: 0.115191 Labeled
epoch 18/50 train loss: 0.115315 Labeled
epoch 18/50 train loss: 0.115425 Labeled
epoch 18/50 train loss: 0.115845 Labeled
epoch 18/50 train loss: 0.115950 Labeled
END Epoch 18/50 Train loss: 0.115825 
Start epoch 19
epoch 19/50 train loss: 0.117476 Labeled
epoch 19/50 train loss: 0.114875 Labeled
epoch 19/50 train loss: 0.113381 Labeled
epoch 19/50 train loss: 0.114382 Labeled
epoch 19/50 train loss: 0.115478 Labeled
epoch 19/50 train loss: 0.115519 Labeled
epoch 19/50 train loss: 0.114615 Labeled
epoch 19/50 train loss: 0.114709 Labeled
epoch 19/50 train loss: 0.114691 Labeled
epoch 19/50 train loss: 0.1150

## FASE DI TESTING

In [ ]:
topk=10
predictions = []
y_true = []

model.eval()

with torch.no_grad():
    step = 0
    for inputs, labels in testDataLoader:

        step += 1
        inputs,label = inputs.view(16,20,3,224,224), labels.to("cuda")
        logps = model(inputs.to(device))
        y_pred = logps

        _, idx = y_pred.topk(topk, dim=1)

        y_pred = torch.zeros_like(y_pred)
        y_pred.scatter_(1, idx, 1)
        predictions.append(y_pred.cpu())

        y_true.append(label.cpu())

y_true, predictions = torch.cat(y_true, axis=0), torch.cat(predictions, axis=0)
report = classification_report(y_true, predictions, 
                               target_names=list(sorted(label_idx.keys())),zero_division=0)
print(report)

                      precision    recall  f1-score   support

                LGBT       0.09      0.49      0.15        61
              action       0.22      0.95      0.36       190
     action_comedies       0.00      0.00      0.00         6
           adventure       0.14      0.64      0.23        74
   alcohol_addiction       0.00      0.00      0.00         6
               alien       0.00      0.00      0.00        12
           animation       0.25      0.71      0.38        21
            aviation       0.00      0.00      0.00        20
        bank_robbery       0.00      0.00      0.00        19
              biopic       0.07      0.46      0.12        57
             cartoon       0.19      0.38      0.25         8
               chase       0.00      0.00      0.00        16
            children       0.00      0.00      0.00        12
              comedy       0.20      0.95      0.33       209
        comedy_drama       0.08      0.48      0.14        65
       